### Getting transactions by using Etherscan API

### 1. Prepare data

In [1]:
folds_num = 4
fold = 0

# create valid unique wallets
df1 = pd.read_csv('data/data_rounds.csv')
list_rounds = [
    '1_gitcoin_gr15',
    '2_unicef',
    '3_fantom',
    '41_gitcoin_climate',
    '42_gitcoin_ethereum',
    '43_gitcoin_oss'
]
df2 = pd.DataFrame({
    'wallet': df1[df1.round_name.isin(list_rounds)].source_wallet.unique()
})
df2 = df2
df2['fold'] = [i % folds_num for i in range(len(df2))]
print(df2.shape)
df2 = df2[df2.fold == fold]
print(df2.shape)

(81387, 2)
(20347, 2)


### 2. Parse transactions

In [2]:
import requests
import pickle
YourApiKeyToken = '' # insert your token

list_wallet  = list()
list_resp = list()

for wallet in tqdm(df2.wallet):
    try:
        url = f'''https://api.etherscan.io/api
           ?module=account
           &action=txlist
           &address={wallet}
           &startblock=0
           &endblock=99999999
           &page=1
           &offset=10000
           &sort=desc
           &apikey={YourApiKeyToken}'''
        url = ''.join([u.strip() for u in url.split('\n')])

        resp = requests.get(url,timeout = 20)
        if resp.status_code != 200:
            print('blocked')
            break
        list_wallet.append(wallet)
        list_resp.append(resp)
    except:
        print(wallet)

# save to pickle
pd.DataFrame({
    'wallet': list_wallet,
    'resp': list_resp,
}).to_pickle(f'data/external/df{fold}.pkl')

100%|██████████████████████████████████████████████████████████████████████████| 20347/20347 [5:06:08<00:00,  1.11it/s]
